In [ ]:
from keras import datasets

(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
train_images =X_train.astype('float32')/255
test_images=X_test.astype('float32')/255

In [ ]:
from keras.utils import to_categorical

train_labels=to_categorical(y_train)
test_labels=to_categorical(y_test)

In [ ]:
val_images=train_images[:10000]
partial_images=train_images[10000:]

val_labels=train_labels[:10000]
partial_labels=train_labels[10000:]

In [ ]:
from keras import layers
from keras import models
from keras.callbacks import EarlyStopping

In [ ]:
def CNN_model( filter1, filter2, filter3, k, activation_fx_1, activation_fx_2, dropout1, dropout2, optimizer, epochs):
  model = models.Sequential()
  model.add(layers.Conv2D(filters = filter1, kernel_size = (k, k), activation = activation_fx_1, input_shape = (32,32,3)))
  model.add(layers.Conv2D(filters = filter1, kernel_size = (k, k), activation = activation_fx_1))
  model.add(layers.MaxPooling2D(2,2))
  model.add(layers.Conv2D(filters = filter2, kernel_size = (k, k), activation = activation_fx_2))
  model.add(layers.Conv2D(filters = filter2, kernel_size = (k, k), activation = activation_fx_2))
  model.add(layers.MaxPooling2D(2,2))
  model.add(layers.Flatten())
  model.add(layers.Dropout(rate = dropout1))
  model.add(layers.Dense(units = filter3, activation = activation_fx_2))
  model.add(layers.Dropout(rate = dropout2))
  model.add(layers.Dense(10, activation= "softmax"))

  model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
  es = EarlyStopping(monitor="val_accuracy", patience = 7)
  model.fit(partial_images, partial_labels, validation_data=(val_images,val_labels), epochs=epochs, batch_size = 100, callbacks = [es], verbose=0)

  return model

In [ ]:
from random import choice
from random import uniform
from numpy.random import randint

In [ ]:
def initialization():
  parameters = {}
  filter1 = choice([32, 64])
  parameters["filter1"] = filter1
  filter2 = choice([64, 128])
  parameters["filter2"] = filter2
  filter3 = choice([128, 256, 512])
  parameters["filter3"] = filter3
  k = choice([3,5])
  parameters["k"] = k
  activation_fx_1 = choice(["relu", "selu", "elu"])
  parameters["activation_fx_1"] = activation_fx_1
  activation_fx_2 = choice(["relu", "selu", "elu"])
  parameters["activation_fx_2"] = activation_fx_2
  dropout1 = round(uniform(0.1, 0.5), 1)
  parameters["dropout1"] = dropout1
  dropout2 = round(uniform(0.1, 0.5), 1)
  parameters["dropout2"] = dropout2
  optimizer = choice(["adamax", "adadelta", "adam", "adagrad"])
  parameters["optimizer"] = optimizer
  epochs = randint(50, 100)
  parameters["epochs"] = epochs
  return parameters

In [ ]:
def generate_population(n):
  population = []
  for i in range(n):
    chromosome = initialization()
    population.append(chromosome)
  return population

In [ ]:
# Fitness evaluation metric: Classification Accuracy
def fitness_evaluation(model):
  metrics = model.evaluate(test_images, test_labels)
  return metrics[1]

In [ ]:
def selection(population_fitness):
  total = sum(population_fitness)
  percentage = [round((x/total) * 100) for x in population_fitness]
  selection_wheel = []
  for pop_index,num in enumerate(percentage):
    selection_wheel.extend([pop_index]*num)
  parent1_ind = choice(selection_wheel)
  parent2_ind = choice(selection_wheel)
  return [parent1_ind, parent2_ind]

In [ ]:
def crossover(parent1, parent2):
  child1 = {}
  child2 = {}

  child1["filter1"] = choice([parent1["filter1"], parent2["filter1"]])
  child1["filter2"] = choice([parent1["filter2"], parent2["filter2"]])
  child1["filter3"] = choice([parent1["filter3"], parent2["filter3"]])

  child2["filter1"] = choice([parent1["filter1"], parent2["filter1"]])
  child2["filter2"] = choice([parent1["filter2"], parent2["filter2"]])
  child2["filter3"] = choice([parent1["filter3"], parent2["filter3"]])

  child1["k"] = choice([parent1["k"], parent2["k"]])
  child2["k"] = choice([parent1["k"], parent2["k"]])

  child1["activation_fx_1"] = parent1["activation_fx_2"]
  child2["activation_fx_1"] = parent2["activation_fx_2"]

  child1["activation_fx_2"] = parent2["activation_fx_1"]
  child2["activation_fx_2"] = parent1["activation_fx_1"]

  child1["dropout1"] = parent1["dropout1"]
  child2["dropout1"] = parent2["dropout1"]

  child1["dropout2"] = parent2["dropout2"]
  child2["dropout2"] = parent1["dropout2"]

  child1["optimizer"] = parent2["optimizer"]
  child2["optimizer"] = parent1["optimizer"]

  child1["epochs"] = parent1["epochs"]
  child2["epochs"] = parent2["epochs"]
  return [child1, child2]


In [ ]:
def mutation(chromosome):
  flag = randint(0,40)
  if flag <= 20:
    chromosome["epochs"] += randint(0, 10)
  return chromosome

In [ ]:
generations = 3
threshold = 90
num_pop = 10

population = generate_population(num_pop)

for generation in range(generations):

  population_fitness = []
  for chromosome in population:
    filter1 = chromosome["filter1"]
    filter2 = chromosome["filter2"]
    filter3 = chromosome["filter3"]
    k = chromosome["k"]
    activation_fx_1 = chromosome["activation_fx_1"]
    activation_fx_2 = chromosome["activation_fx_2"]
    dropout1 = chromosome["dropout1"]
    dropout2 = chromosome["dropout2"]
    optimizer = chromosome["optimizer"]
    epochs = chromosome["epochs"]

    try:
      model = CNN_model(filter1, filter2, filter3, k, activation_fx_1, activation_fx_2, dropout1, dropout2, optimizer, epochs)
      acc = fitness_evaluation(model)
      print("Parameters: ", chromosome)
      print("Accuracy: ", round(acc,3))
    except:
      acc=0
      print("Parameters: ", chromosome)
      print("Invalid parameters - Build fail")

    population_fitness.append(acc)

  parents_ind = selection(population_fitness)
  parent1 = population[parents_ind[0]]
  parent2 = population[parents_ind[1]]

  children = crossover(parent1, parent2)
  child1 = mutation(children[0])
  child2 = mutation(children[1])

  population.append(child1)
  population.append(child2)

  print("Generation ", generation+1," Outcome: ")
  if max(population_fitness) >= threshold:
    print("Obtained desired accuracy: ", max(population_fitness))
    break
  else:
    print("Maximum accuracy in generation {} : {}".format(generation+1, max(population_fitness)))

  first_min = min(population_fitness)
  first_min_ind = population_fitness.index(first_min)
  population.remove(population[first_min_ind])
  second_min = min(population_fitness)
  second_min_ind = population_fitness.index(second_min)
  population.remove(population[second_min_ind])